# Synthetic Bias Diagnostics (Acceptance Loop)
Inspect whether selection bias **grows or stabilizes** during the acceptance loop, and summarize final exported datasets (`Da`, `Dr`, `H`).

**Inputs**: a generated run folder containing:
- `Da.csv`, `Dr.csv`, `H.csv`
- `meta.json`
- `snapshots/iter_*.json`


In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- Set this to your generated run folder ----
RUN_DIR = Path("synthetic_runs/REPLACE_WITH_RUN_ID")  # e.g., synthetic_runs/2026-01-22_001
assert RUN_DIR.exists(), f"RUN_DIR not found: {RUN_DIR.resolve()}"

DA_PATH = RUN_DIR / "Da.csv"
DR_PATH = RUN_DIR / "Dr.csv"
H_PATH  = RUN_DIR / "H.csv"
META_PATH = RUN_DIR / "meta.json"
SNAP_DIR = RUN_DIR / "snapshots"

for p in [DA_PATH, DR_PATH, H_PATH, META_PATH, SNAP_DIR]:
    assert p.exists(), f"Missing: {p}"

meta = json.loads(META_PATH.read_text())
print("meta keys:", sorted(meta.keys())[:20], "...")


In [ ]:
Da = pd.read_csv(DA_PATH)
Dr = pd.read_csv(DR_PATH)
H  = pd.read_csv(H_PATH)

print("Da:", Da.shape)
print("Dr:", Dr.shape)
print("H :", H.shape)

assert "y" in Da.columns and "y" in H.columns, "Da/H must contain y"
assert "y" not in Dr.columns, "Dr must NOT contain y (unlabeled rejects)"

print(f"Bad rate: Da={Da['y'].mean():.4f}, H={H['y'].mean():.4f}")


In [ ]:
# Load snapshot diagnostics
snap_files = sorted(SNAP_DIR.glob("iter_*.json"))
assert len(snap_files) > 0, "No snapshot json files found."

snaps = [json.loads(f.read_text()) for f in snap_files]
snap_df = pd.DataFrame(snaps).sort_values("iter")

snap_df.head()


In [ ]:
# Plot: acceptance growth and bad rate over iterations
plt.figure()
plt.plot(snap_df["iter"], snap_df["n_accepts"])
plt.xlabel("Iteration")
plt.ylabel("Cumulative accepts (n_accepts)")
plt.title("Acceptance growth over iterations")
plt.show()

plt.figure()
plt.plot(snap_df["iter"], snap_df["bad_rate_accepts"])
plt.xlabel("Iteration")
plt.ylabel("Bad rate among accepts")
plt.title("Bad rate among accepts vs iteration")
plt.show()

if "auc_on_H" in snap_df.columns:
    plt.figure()
    plt.plot(snap_df["iter"], snap_df["auc_on_H"])
    plt.xlabel("Iteration")
    plt.ylabel("AUC on H (loop model)")
    plt.title("Model AUC on H vs iteration (during loop)")
    plt.show()


In [ ]:
# Drift proxy: KS(Da vs H) per numeric feature (final snapshot)
from scipy.stats import ks_2samp

feat_cols = [c for c in Da.columns if c != "y"]
num_cols = [c for c in feat_cols if pd.api.types.is_numeric_dtype(Da[c])]

ks_stats = []
for c in num_cols:
    a = Da[c].to_numpy()
    h = H[c].to_numpy()
    a = a[~np.isnan(a)]
    h = h[~np.isnan(h)]
    if len(a) == 0 or len(h) == 0:
        continue
    ks = ks_2samp(a, h).statistic
    ks_stats.append((c, ks))

ks_df = pd.DataFrame(ks_stats, columns=["feature", "ks_Da_vs_H"]).sort_values("ks_Da_vs_H", ascending=False)
ks_df.head(15)


In [ ]:
# Plot top drifting features (KS)
top = ks_df.head(15)
plt.figure(figsize=(10, 5))
plt.bar(range(len(top)), top["ks_Da_vs_H"].to_numpy())
plt.xticks(range(len(top)), top["feature"].to_list(), rotation=75, ha="right")
plt.ylabel("KS statistic (Da vs H)")
plt.title("Top 15 drifting features (final Da vs H)")
plt.tight_layout()
plt.show()

print("Median KS:", float(ks_df["ks_Da_vs_H"].median()))
print("Max KS:", float(ks_df["ks_Da_vs_H"].max()))


## Interpreting results
- **n_accepts** should grow roughly linearly to your target (e.g., ~40k).
- **bad_rate_accepts** should generally drop below **bad_rate(H)** due to selection.
- **auc_on_H** (if tracked) often improves early then stabilizes.
- **KS drift** should be non-trivial; late iterations often stabilize if the policy stabilizes.

If things look unstable:
- increase policy noise slightly (`σ_policy`)
- increase initial seed accepts
- reduce XGBoost complexity (depth/estimators) inside loop
- reduce accept quota per iter
